## Load Climate Classification
Here we generate the area-averaged climate classifications for each dam.

In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from tools import save_hdf
from itertools import compress
from functools import reduce
from scipy import stats

In [2]:
# Load Degree of Regulation (DOR)
dor = pd.read_hdf('./data/new_dor.hdf')      # The order is sorted during the process
dam_dor = dor.loc[dor.DOR2 <= 0, 'GRAND_ID']
# Load Dam Inflow data from SUTD
dfFlowDams = pd.read_hdf('./data/dfFlowDams.hdf')
ind_dams = np.load('./data/ind_dams.npz')['ind_dams']
damList = ind_dams[0,:]
ind_dams = ind_dams[1,:]
ndam = len(damList)

ValueError: Invalid frequency: b'ccopy_reg\n_reconstructor\np0\n(cpandas.tseries.offsets\nMonthEnd\np1\nc__builtin__\nobject\np2\nNtp3\nRp4\n(dp5\nVn\np6\nL1L\nsVnormalize\np7\nI00\nsV_cache\np8\n(dp9\nVfreqstr\np10\nVM\np11\nssb.'

### Countries and Continents

In [ ]:
# Load 1593 GranD dam shapefile
dams = gpd.read_file('./data/granddams_eval.shp')
dams = dams.drop(dams.columns[1:-1], axis=1)
dams = dams.set_index('GRAND_ID').loc[damList].reset_index(drop=False)
# Load world base map (exclude Antarctica)
world = gpd.read_file('./data/world_countries_without_antarctica.shp').drop(['ObjectID','ISO_CC','Land_Type','Land_Rank','COUNTRYAFF'], axis=1)
dams = gpd.sjoin(dams, world, how="left", op='within')
# Manual editing
dams.loc[dams['GRAND_ID'] == 6841, ['COUNTRY','CONTINENT']] = ['France', 'Europe']
dams.loc[dams['GRAND_ID'] == 3043, ['COUNTRY','CONTINENT']] = ['Liberia', 'Africa']
dams.loc[dams['GRAND_ID'] == 1916, ['COUNTRY','CONTINENT']] = ['United States', 'North America']
dams.loc[dams['GRAND_ID'] == 6841, ['COUNTRY','CONTINENT']] = ['United States', 'North America']
dams['CONTINENT'] = dams['CONTINENT'].replace({'Australia': 'Oceania'})
dams.head()

### (a) Hydrological Climate Classification
The [HCC (Knoben et al., 2018)]((https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2018WR022913)) data is downloaded from [repository](https://data.bris.ac.uk/data/dataset/16ctquxqxk46h2v61gz7drcdz3).

In [ ]:
# Load the Hydrological Climate Classification (HCC)
filn_hcc = './data/hydroclimate_classification/ClimateClassification_mainMap_geoReferenced.tif'
hcc = rasterio.open(filn_hcc).read(range(1,4))    # ["aridity", "seasonality", "snow"]
hcc.shape

### (b) The Köppen-Geiger Climate Classification
- (1) [Original KGC (Kottek et al., 2006)](http://koeppen-geiger.vu-wien.ac.at/present.htm)
- (2) [Updated KGC (Peel et al., 2007)](https://www.hydrol-earth-syst-sci.net/11/1633/2007/)

In [ ]:
# (1) Original KGC (ASCII format)
kgc1_asc = pd.read_excel('./data/koppen_geiger_classification/Koeppen_Geiger_ASCII.xlsx')
code1 = np.unique(kgc1_asc['Cls'])
code1 = np.vstack([np.arange(1,len(code1)+1), code1]).transpose()
code1 = pd.DataFrame(code1, columns=['Number','Code'])
kgc1_asc = pd.merge(kgc1_asc, code1, how='inner', left_on='Cls', right_on='Code').drop('Code', axis=1)
# - Converts to Raster format
fn_kgc1 = './data/koppen_geiger_classification/Koeppen_Geiger_ASCII.tif'
if not os.path.exists(fn_kgc1):
    lat = np.linspace(89.75, -89.75, 360)
    lon = np.linspace(-179.75, 179.75, 720)
    lon, lat = np.meshgrid(lon, lat)
    kgc1 = np.full(lon.shape, np.nan)
    for index, row in kgc1_asc.iterrows():
        targ = (lat == row['Lat']) & (lon == row['Lon'])
        kgc1[targ] = row['Number']
    with rasterio.open(filn_hcc) as src:
        ras_meta = src.profile
    with rasterio.open(fn_kgc1, 'w', **ras_meta) as dst:
        dst.write(kgc1, 1)
kgc1 = rasterio.open(fn_kgc1).read(1)

# (2) Updated KGC
# Import colormap
rgb_code = pd.read_excel('./data/koppen_geiger_classification/code_rgb.xlsx', index_col=0)
cmap = dict()
for i, em in rgb_code.iterrows():
    cmap[i] = tuple(em[['Red','Green','Blue']])
# Save GeoTiff file
# - Original 0.01 degree
with rasterio.open('./data/koppen_geiger_classification/world_koppen.tif') as src:
    kgc2_raw = data = src.read(1)
    profile = src.profile
    with rasterio.open('./data/koppen_geiger_classification/world_koppen_modified_0_01d.tif','w', **profile) as dst:
        dst.write(data, 1)
        dst.write_colormap(1, cmap)
# - Resampled 0.05 degree
with rasterio.open('./data/koppen_geiger_classification/kgc_aligned.tif') as src:
    kgc2_raw = data = src.read(1)
    profile = src.profile
    profile['nodata'] = 255
    with rasterio.open('./data/koppen_geiger_classification/world_koppen_modified_0_05d.tif','w', **profile) as dst:
        dst.write(data, 1)
        dst.write_colormap(1, cmap)
        
# The 0.1 degree data is upscaled (wrapped) to 0.5 degree resolution using "Mode" resampling method.
kgc2 = rasterio.open('./data/koppen_geiger_classification/kgc_aligned.tif').read(1)
code2 = pd.read_excel('./data/koppen_geiger_classification/code_rgb.xlsx')

### Extract upstream grids for each dam

In [3]:
# Load streamflow network (DDM30)
with rasterio.open('./data/DDM30_fdir_rec.tif') as src:
    fdir = src.read(1)
    fdir = np.vstack((np.zeros([12,720]), fdir, np.zeros([68,720])))
    
# Grid Index of DDM30
gdix = np.arange(360*720).reshape([360,720])
# Grid index of downstream
down = np.full(gdix.shape, np.nan)
down[fdir == 1] = gdix[fdir == 1] + 1        # East
down[fdir == 2] = gdix[fdir == 2] + 721      # SouthEast
down[fdir == 4] = gdix[fdir == 4] + 720      # South
down[fdir == 8] = gdix[fdir == 8] + 719      # SouthWest
down[fdir == 16] = gdix[fdir == 16] - 1      # West
down[fdir == 32] = gdix[fdir == 32] - 721    # NorthWest
down[fdir == 64] = gdix[fdir == 64] - 720    # North
down[fdir == 128] = gdix[fdir == 128] - 719  # NorthEast
# DataFrame of stream index
gdix = pd.DataFrame(data=np.vstack((gdix.flatten(),down.flatten())).transpose(),
                    columns=['up','down'])
gdix.index.name='index'

# Algorithm of identifying upstream grid indices
def CumUpStreamGrid(init, gdix):
    total = [init]
    incr = gdix.loc[np.isin(gdix['down'], init), 'up'].values
    total.extend(incr)
    while len(incr) != 0:
        incr = gdix.loc[np.isin(gdix['down'], incr), 'up'].values
        total.extend(incr)
    return list(np.array(total).astype(int))

In [4]:
# Generate upstream grid indices for each dam
upgrid = {}
for i, did in enumerate(damList):
    upgrid.update({did: CumUpStreamGrid(ind_dams[i], gdix)})     
    
# Validated as no zero length and nan values
length = np.array([ len(gdx) for _, gdx in upgrid.items()])
assert np.sum(length == 0) == 0
num_nan = np.array([ np.isnan(np.array(gdx)).sum() for _, gdx in upgrid.items()])
assert np.sum(num_nan > 0) == 0

NameError: name 'damList' is not defined

### Climate Classification Values of each dams
- Averaged HCC indices
- Mode (most frequent) KGC classes

In [5]:
# Reshaping map to linear data
hcc = hcc.reshape((3,360*720))
kgc1 = kgc1.reshape((1,360*720))
kgc2 = kgc2.reshape((1,360*720))
# Select classification values
# - For HCC, we use averaged values
# - For KGC, we use mode (most frequent) values
hcc_extr = np.zeros((ndam,3))
kgc_extr = np.zeros((ndam,2))
for i, did in enumerate(damList):
    hcc_extr[i,:] = hcc[:, upgrid[did]].mean(1)
    kgc_extr[i,0] = stats.mode(kgc1.flatten()[upgrid[did]])[0]
    kgc_extr[i,1] = stats.mode(kgc2.flatten()[upgrid[did]])[0]
cclass = pd.DataFrame(data=np.hstack((hcc_extr, kgc_extr)), index=damList, columns=["aridity", "seasonality", "snow", "kgc1", "kgc2"])
cclass.index.name = 'GRAND_ID'

# ***MANUAL EDITING***
# - Missing KGC2 (GRAND_ID: 5818)
cclass.loc[cclass['kgc2'] == 255, 'kgc2'] = 1
# # - High elevation (see Readme.pdf of Peel et al. (2007))
# cclass.loc[cclass['kgc2'] == 31, 'kgc2'] = 29
# cclass.loc[cclass['kgc2'] == 32, 'kgc2'] = 30

# Replace KGC numbers to class names
for number, em in code1.set_index('Number').iterrows():
    cclass.loc[cclass['kgc1'] == number,'kgc1'] = em['Code']
for number, em in code2[['Number','Code']].set_index('Number').iterrows():
    cclass.loc[cclass['kgc2'] == number,'kgc2'] = em['Code']

# Merge country and continent
cclass = cclass.merge(dams[['GRAND_ID', 'CONTINENT', 'COUNTRY']], on='GRAND_ID', how='inner').set_index('GRAND_ID')
# - One reservoir falls in the lake
cclass.loc[2078, ['CONTINENT', 'COUNTRY']] = cclass.loc[2087, ['CONTINENT', 'COUNTRY']]

# Saving
save_hdf('./data/climate_classification.hdf', cclass)
cclass.to_excel('./data/dam_climate_classification_201201.xlsx')
cclass.head()

NameError: name 'hcc' is not defined

### Jia's questions

In [9]:
# Frequency of classes
np.unique(kgc1[~np.isnan(kgc1)])
c, n = np.unique(kgc2_raw[~np.isnan(kgc2_raw)],return_counts=True)
# np.vstack([c,n]).T

In [10]:
gdfDam = gpd.read_file('./data/granddams_eval.shp')
map_cclass = pd.merge(gdfDam, cclass, on='GRAND_ID')
map_cclass.to_file('./data/map_climate_classification.shp')

In [11]:
cclass

,aridity,seasonality,snow,kgc1,kgc2,CONTINENT,COUNTRY
GRAND_ID,,,,,,,
6,0.394127,0.727136,0.400318,Dfc,Dfc,North America,Canada
24,0.093695,0.645148,0.272423,Cfc,Cfb,North America,Canada
25,0.093695,0.645148,0.272423,Cfc,Cfb,North America,Canada
27,0.188873,0.725226,0.335959,Cfb,Csb,North America,Canada
31,0.099993,0.702323,0.000000,Cfb,Cfb,North America,Canada
...,...,...,...,...,...,...,...
10027,0.644849,0.578219,0.065697,ET,ET,Asia,China
10028,0.632586,0.577068,0.098514,ET,Dwb,Asia,China
2078,0.421002,0.554055,0.253935,Dfb,Dfb,North America,United States


In [12]:
cclass.loc[10024]

aridity        0.750085
seasonality    0.459232
snow           0.098762
kgc1                 ET
kgc2                 ET
CONTINENT          Asia
COUNTRY           China
Name: 10024, dtype: object

In [15]:
kgc2.flatten()[upgrid[10024]]

array([12, 15, 15, 12, 22, 22, 22, 22, 22, 22,  7, 22,  7,  7,  7,  7, 12,
        7, 12, 22, 22, 12, 22, 22, 22, 22, 22, 23, 23, 23, 23, 31, 31, 31,
       31, 31, 31, 31, 23, 23,  7,  7, 31, 31,  7, 31, 31, 31, 31, 31, 31,
       31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31,
       31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31], dtype=uint8)